## Unsloth 활용 LLM 파인 튜닝 

In [1]:
from unsloth import FastLanguageModel

import torch
max_seq_length = 2048 
dtype = None 
load_in_4bit = True

#Unsloth에서 제공하는 Llama기반 
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.7.3: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 4. Max memory: 23.553 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


/home/moon/miniconda3/envs/fine/lib/python3.11/site-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, #LoRA 랭크 값(LoRA가 학습할 수 있는 정보량)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, # LoRA 학습에 ㅎ사용되는 스케일링 계수(r과 같거나 2배 정도로 설정), 학습 강도를 조절(값이 크면 더 많이 학습)
    lora_dropout = 0, # 드롭아웃 비율(학습 중 일부 뉴련을 무작위로 끄는 확률)
    bias = "none",    # bias 사용 여부
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # 그래디언트 체크포인팅 활성화 여부(true, False, unsloth)
    random_state = 3407, # 랜덤시드값(재현 가능한 결과를 위한 설정)
    use_rslora = False,  # Rank-Stabilized LoRA 사용 여부 
    loftq_config = None, # And LoftQ( LoRA 행렬을 weight의 상위 특이벡터로 초기화 → 정확도 향상 가능)
)

Unsloth 2025.7.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [3]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [5]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("moon301/lh_announcement", split = "korean_v1")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map: 100%|██████████| 160/160 [00:00<00:00, 14057.16 examples/s]


In [6]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, 
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=20): 100%|██████████| 160/160 [00:04<00:00, 35.58 examples/s]


In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 160 | Num Epochs = 12 | Total steps = 60
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.229400
2,3.240700
3,3.193500
4,3.057700
5,2.773300
6,2.383600
7,1.957300
8,1.576700
9,1.328600
10,1.069900


In [8]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "다음 문서의 내용에 대해 질문하세요", # instruction
        "이 공고문은 어떤 주택단지의 예비입주자를 모집하고 있나요?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n다음 문서의 내용에 대해 질문하세요\n\n### Input:\n이 공고문은 어떤 주택단지의 예비입주자를 모집하고 있나요?\n\n### Response:\n강남에버시움(서울강남3)과 서울서초3 국민임대주택의 예비입주자를 모집하고 있습니다.<|end_of_text|>']

In [25]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "다음 문서의 내용에 대해 질문하세요", # instruction
        "이 공고문의 1순위부터 3순위까지 접수일을 알려주세요", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
다음 문서의 내용에 대해 질문하세요

### Input:
이 공고문의 1순위부터 3순위까지 접수일을 알려주세요

### Response:


2025년 3월 14일, 2025년 3월 17일, 2025년 3월 19일입니다.<|end_of_text|>


In [ ]:
model.push_to_hub("moon301/llama-3.1-8b-finetuned-lh-announcement", token = "") # Online saving
tokenizer.push_to_hub("moon301/llama-3.1-8b-finetuned-lh-announcement", token = "") # Online saving

100%|██████████| 1/1 [00:14<00:00, 14.93s/it]


Saved model to https://huggingface.co/moon301/llama-3.1-8b-finetuned-lh-announcement


100%|██████████| 1/1 [00:02<00:00,  2.59s/it]


In [ ]:
model.save_pretrained_gguf("model", tokenizer,)
model.push_to_hub_gguf("moon301/model", tokenizer, token = "")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 137.47 out of 188.38 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:00<00:00, 53.35it/s]


Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...


RuntimeError: Unsloth: The file 'llama.cpp/llama-quantize' or `llama.cpp/quantize` does not exist.
We've also double checked the building directory under 'llama.cpp/build/bin/'.
But we expect this file to exist! Check if the file exists under llama.cpp and investigate the building process of llama.cpp (make/cmake)!

In [ ]:
from huggingface_hub import login
from transformers import AutoTokenizer

# Huggingface 토큰 로그인
login("")  # 한번만 실행

# 저장할 디렉토리 이름
repo_name = "moon301/llama-3.1-8b-finetuned-lh-announcement"  # 예: "shyeon31/kanana-nano-2.1b-finetuned"

# 모델 저장
trainer.model.save_pretrained(repo_name, push_to_hub=True)

# tokenizer도 같이 저장
tokenizer.save_pretrained(repo_name, push_to_hub=True)


100%|██████████| 1/1 [00:02<00:00,  2.61s/it]


('moon301/llama-3.1-8b-finetuned-lh-announcement/tokenizer_config.json',
 'moon301/llama-3.1-8b-finetuned-lh-announcement/special_tokens_map.json',
 'moon301/llama-3.1-8b-finetuned-lh-announcement/tokenizer.json')